# Introduction to using ParticleSpy

This is an example notebook for performing particle analysis using ParticleSpy.

This notebook takes a HAADF image of nanoparticles, allows the user to determine the best segmentation method and then performs analysis, producing a plot of particle areas.

Below, we load an image using Hyperspy.

In [ ]:
import hyperspy.api as hs
import particlespy.api as ps
import trackpy as tp
import matplotlib.pyplot as plt

filename = 'SiO2 HAADF Image.hspy'
haadf = hs.load(filename)

%matplotlib notebook
haadf.plot()

ParticleSpy works by defining parameters to run segmentation. This is done through the parameters object, which can be loaded with default values.

In [ ]:
params = ps.parameters()
params.generate()

Let's take a look at the default segmentation parameters.

In [ ]:
params.segment

The default is a simple Otsu threshold with no other options selected. We could apply these parameters to segment the particles but we don't know how well these will work on our individual images. In order to check this out, we can use ParticleSpy's segmentation GUI. Experiment with changing the threshold algorithm and the application of watershedding to obtain the optimum labels. Once happy, click "Get Params" to print the current parameters.

<span style="color:red">WARNING: Don't run this if on Google Cloud Platform!</span>

In [ ]:
ps.seg_ui(haadf)

Next, we use the parameters saved when last pressing Update in the GUI to construct a parameter dictionary.

In [ ]:
params.load()
params.store['store_im'] = True

If running a notebook on a remote server there is an alternative method to check segmentation parameters using the process function (however, this isn't included in the api).

In [ ]:
import particlespy
seg = particlespy.segptcls.process(haadf,params)
plt.figure()
plt.imshow(seg)

Once we're happy with the segmentation parameters, we run the particle_analysis function to segment our image and provide a list of all the particles.

In [ ]:
particles = ps.particle_analysis(haadf,params)

Let's interrogate the particle data a little bit. From an initial image we get a number of particle properties calculated automatically.

In [ ]:
print(particles.list[0].properties)

It is then possible to plot a histogram of particle area.

In [ ]:
particles.plot()

It is also possible to plot a scatter plot of two properties. In this instance, plotting 'intensity' vs area.

In [ ]:
particles.plot(['intensity','area'])

## ParticleSpy with EDS data

If you have EDS data collected simultaneously with an image, it is possible to extract maps, spectra and the composition from each segmented particle. Firstly, load the EDS spectrum image.

In [ ]:
eds_filename = 'SiO2 EDS Spectrum Image.hspy'
eds = hs.load(eds_filename)

Next, put the HAADF image and EDS spectrum image together in a list to make one acquisition.

In [ ]:
ac = [haadf.isig[0:32,0:32],eds.inav[0:32,0:32]]

At this point, we need to set the parameters for the EDS analysis. This can be done by taking our previously used parameters object adding parameters with generate_eds().

In [ ]:
params.generate_eds(eds_method='CL',elements=['Si','O'], factors=[1.,1.3],store_maps=True)

Now, we need to run ParticleAnalysis again in order to get the accompanying EDS data.

In [ ]:
particles2 = ps.particle_analysis(ac,params)

The extracted maps of each element can be accessed from the particle list from maps['element'].

In [ ]:
particles2.list[1].maps['Si'].plot()

The composition of each particle can be accessed from the particle list using list[particle].composition.

In [ ]:
particles2.list[0].composition

We can plot a radial profile of the particle intensity from both the image and our elemental maps.

In [ ]:
rp = ps.radial_profile(particles2.list[1],['Image'],plot=True)

In [ ]:
rp2 = ps.radial_profile(particles2.list[1],['Si','O'],plot=True)

## ParticleSpy with time-series data

ParticleSpy can also be used to track properties through a time-series of images. Let's see that in action on a short series of only 10 frames.

In [ ]:
series = hs.load('particle_series.dm4')

In [ ]:
series.plot()

We'll define some new parameters that work well for this data.

In [ ]:
params_series = ps.parameters()

In [ ]:
params_series.generate(threshold='mean',min_size=200,gaussian=3)

For particle series, there is a modified ParticleAnalysis function called ParticleAnalysisSeries that deals with multiple frames.

In [ ]:
particles_series = ps.particle_analysis_series(series, params_series)

In [ ]:
particles_series.list[0].properties

Now we have a list of particles, with their properties attached including the frame number. We can now pass them through another function that uses the trackpy package to track the same particles through a time series.

In [ ]:
series_test = ps.time_series_analysis(particles_series)

The returned series_test is a pandas dataframe, which is the standard object that is used in trackpy.

We can use trackpy to do cool things like plot the trajectory of all particles in the series. Note: while the axes say they are in pixels, they are actually in nm in this case! (I have just noticed this and will fix)

In [ ]:
plt.figure()
tp.plot_traj(series_test)

Trackpy also lets you correct for any systematic drift in the series.

In [ ]:
drift = tp.compute_drift(series_test)
drift.plot()

In [ ]:
tm = tp.subtract_drift(series_test.copy(), drift)
tp.plot_traj(tm)

Using the particlespy workflow for time-series data, you also have any of the properties specified in the pandas dataframe and you can plot these as a function of frame number.

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
for index, particle in tm.groupby('particle'):
    plt.plot(particle['frame'], particle['area'], label=index)